# 信賴區間

## RF + Bootstrap CI

In [34]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

# === 載入原始資料 ===
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與商務艙
economy_data = data[data['艙等'] == '經濟艙'].copy()
business_data = data[data['艙等'] == '商務艙'].copy()

# One-hot encoding
cat_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
            '航空公司', '航空聯盟', '機型分類', '是否過夜', '是否為平日', '機型', '假期', 'Region']
economy_data = pd.get_dummies(economy_data, columns=cat_cols, drop_first=True)
business_data = pd.get_dummies(business_data, columns=cat_cols, drop_first=True)

# 取共同欄位
common_cols = list(set(economy_data.columns) & set(business_data.columns))
economy_data = economy_data[common_cols]
business_data = business_data[common_cols]

# 標準化數值變數
num_cols = ['飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_data[num_cols] = scaler.fit_transform(economy_data[num_cols])
business_data[num_cols] = scaler.transform(business_data[num_cols])

# 建模欄位
target_col = '最低價格_log'
feature_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空聯盟_', '機型分類_', '假期_', '是否為平日_']
eco_features = [col for col in economy_data.columns if any(k in col for k in feature_keywords)] + num_cols
biz_features = [col for col in business_data.columns if any(k in col for k in feature_keywords)] + num_cols

# Bootstrap 預測區間（95% + 99%）與信心評分
def bootstrap_prediction_ci_rf(X, y_log, label="economy", n_iterations=100):
    X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=123)
    y_test = np.exp(y_test_log.values)
    all_log_preds = []

    for i in tqdm(range(n_iterations), desc=f"Bootstrap RF ({label})"):
        X_resample, y_resample = resample(X_train, y_train_log, replace=True, random_state=100+i)
        model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=100+i)
        model.fit(X_resample, y_resample)
        pred_log = model.predict(X_test)
        all_log_preds.append(pred_log)

    all_log_preds = np.array(all_log_preds)
    mean_log_pred = np.mean(all_log_preds, axis=0)
    mean_pred = np.exp(mean_log_pred)

    # CI in log space
    ci_95_lower_log = np.percentile(all_log_preds, 2.5, axis=0)
    ci_95_upper_log = np.percentile(all_log_preds, 97.5, axis=0)
    ci_99_lower_log = np.percentile(all_log_preds, 0.5, axis=0)
    ci_99_upper_log = np.percentile(all_log_preds, 99.5, axis=0)

    # exp back
    ci_95_lower = np.exp(ci_95_lower_log)
    ci_95_upper = np.exp(ci_95_upper_log)
    ci_99_lower = np.exp(ci_99_lower_log)
    ci_99_upper = np.exp(ci_99_upper_log)

    # 是否落在區間
    cover_95 = (y_test >= ci_95_lower) & (y_test <= ci_95_upper)
    cover_99 = (y_test >= ci_99_lower) & (y_test <= ci_99_upper)

    # 信心評估：與 CI 中心距離佔區間比例
    ci_center = (ci_95_upper + ci_95_lower) / 2
    ci_width = ci_95_upper - ci_95_lower
    confidence_score = 1 - np.abs(y_test - ci_center) / (ci_width + 1e-8)
    confidence_score = np.clip(confidence_score, 0, 1)

    # 統計指標
    mse = mean_squared_error(y_test, mean_pred)
    r2 = r2_score(y_test, mean_pred)
    cover_95_ratio = cover_95.mean()
    cover_99_ratio = cover_99.mean()

    print(f"\n📊 [{label}] RF 統計指標：")
    print(f"   MSE         = {mse:.2f}")
    print(f"   R^2         = {r2:.4f}")
    print(f"   95% 覆蓋率   = {cover_95_ratio:.2%}")
    print(f"   99% 覆蓋率   = {cover_99_ratio:.2%}\n")

    df_result = pd.DataFrame({
        "實際價格": y_test,
        "預測值": mean_pred,
        "CI95下限": ci_95_lower,
        "CI95上限": ci_95_upper,
        "CI99下限": ci_99_lower,
        "CI99上限": ci_99_upper,
        "CI95長度": ci_95_upper - ci_95_lower,
        "CI99長度": ci_99_upper - ci_99_lower,
        "是否落在CI95": cover_95.astype(int),
        "是否落在CI99": cover_99.astype(int),
        "信心分數": confidence_score
    })

    output_dir = "/Users/yuchingchen/Documents/專題/ci/ci_data"
    os.makedirs(output_dir, exist_ok=True)
    df_result.to_csv(f"{output_dir}/short_rf_bootstrap_ci_{label}.csv", index=False)
    print(f"✅ [{label}] RF Bootstrap CI 結果已儲存！")

# 執行經濟艙與商務艙 RF 模型
bootstrap_prediction_ci_rf(economy_data[eco_features], economy_data[target_col], label="economy")
bootstrap_prediction_ci_rf(business_data[biz_features], business_data[target_col], label="business")

Bootstrap RF (economy): 100%|██████████| 100/100 [00:21<00:00,  4.68it/s]



📊 [economy] RF 統計指標：
   MSE         = 3116748.31
   R^2         = 0.7605
   95% 覆蓋率   = 42.75%
   99% 覆蓋率   = 52.10%

✅ [economy] RF Bootstrap CI 結果已儲存！


Bootstrap RF (business): 100%|██████████| 100/100 [00:13<00:00,  7.24it/s]


📊 [business] RF 統計指標：
   MSE         = 9983004.19
   R^2         = 0.7509
   95% 覆蓋率   = 39.96%
   99% 覆蓋率   = 49.78%

✅ [business] RF Bootstrap CI 結果已儲存！


## XGB + Bootstrap CI

In [35]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

# === 載入原始資料 ===
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與商務艙
economy_data = data[data['艙等'] == '經濟艙'].copy()
business_data = data[data['艙等'] == '商務艙'].copy()

# One-hot encoding
cat_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
            '航空公司', '航空聯盟', '機型分類', '是否過夜', '是否為平日', '機型', '假期', 'Region']
economy_data = pd.get_dummies(economy_data, columns=cat_cols, drop_first=True)
business_data = pd.get_dummies(business_data, columns=cat_cols, drop_first=True)

# 取共同欄位
common_cols = list(set(economy_data.columns) & set(business_data.columns))
economy_data = economy_data[common_cols]
business_data = business_data[common_cols]

# 標準化數值變數
num_cols = ['飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_data[num_cols] = scaler.fit_transform(economy_data[num_cols])
business_data[num_cols] = scaler.transform(business_data[num_cols])

# 建模欄位
target_col = '最低價格_log'
feature_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空聯盟_', '機型分類_', '假期_', '是否為平日_']
eco_features = [col for col in economy_data.columns if any(k in col for k in feature_keywords)] + num_cols
biz_features = [col for col in business_data.columns if any(k in col for k in feature_keywords)] + num_cols

# Bootstrap 預測區間（95% + 99%）與信心評分
def bootstrap_prediction_ci(X, y_log, label="economy", n_iterations=100):
    X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=123)
    y_test = np.exp(y_test_log.values)
    all_log_preds = []

    for i in tqdm(range(n_iterations), desc=f"Bootstrap ({label})"):
        X_resample, y_resample = resample(X_train, y_train_log, replace=True, random_state=100+i)
        model = XGBRegressor(n_estimators=100, n_jobs=-1, random_state=100+i)
        model.fit(X_resample, y_resample)
        pred_log = model.predict(X_test)
        all_log_preds.append(pred_log)

    all_log_preds = np.array(all_log_preds)
    mean_log_pred = np.mean(all_log_preds, axis=0)
    mean_pred = np.exp(mean_log_pred)

    # CI in log space
    ci_95_lower_log = np.percentile(all_log_preds, 2.5, axis=0)
    ci_95_upper_log = np.percentile(all_log_preds, 97.5, axis=0)
    ci_99_lower_log = np.percentile(all_log_preds, 0.5, axis=0)
    ci_99_upper_log = np.percentile(all_log_preds, 99.5, axis=0)

    # exp back
    ci_95_lower = np.exp(ci_95_lower_log)
    ci_95_upper = np.exp(ci_95_upper_log)
    ci_99_lower = np.exp(ci_99_lower_log)
    ci_99_upper = np.exp(ci_99_upper_log)

    # 是否落在區間
    cover_95 = (y_test >= ci_95_lower) & (y_test <= ci_95_upper)
    cover_99 = (y_test >= ci_99_lower) & (y_test <= ci_99_upper)

    # 信心評估：與 CI 中心距離佔區間比例
    ci_center = (ci_95_upper + ci_95_lower) / 2
    ci_width = ci_95_upper - ci_95_lower
    confidence_score = 1 - np.abs(y_test - ci_center) / (ci_width + 1e-8)
    confidence_score = np.clip(confidence_score, 0, 1)

    # 統計指標
    mse = mean_squared_error(y_test, mean_pred)
    r2 = r2_score(y_test, mean_pred)
    cover_95_ratio = cover_95.mean()
    cover_99_ratio = cover_99.mean()

    print(f"\n📊 [{label}] 統計指標：")
    print(f"   MSE         = {mse:.2f}")
    print(f"   R^2         = {r2:.4f}")
    print(f"   95% 覆蓋率   = {cover_95_ratio:.2%}")
    print(f"   99% 覆蓋率   = {cover_99_ratio:.2%}\n")

    df_result = pd.DataFrame({
        "實際價格": y_test,
        "預測值": mean_pred,
        "CI95下限": ci_95_lower,
        "CI95上限": ci_95_upper,
        "CI99下限": ci_99_lower,
        "CI99上限": ci_99_upper,
        "CI95長度": ci_95_upper - ci_95_lower,
        "CI99長度": ci_99_upper - ci_99_lower,
        "是否落在CI95": cover_95.astype(int),
        "是否落在CI99": cover_99.astype(int),
        "信心分數": confidence_score
    })

    output_dir = "/Users/yuchingchen/Documents/專題/ci/ci_data"
    os.makedirs(output_dir, exist_ok=True)
    df_result.to_csv(f"{output_dir}/short_xgb_bootstrap_ci_{label}.csv", index=False)
    print(f"✅ [{label}] Bootstrap CI 結果已儲存！")

# 執行經濟艙與商務艙
bootstrap_prediction_ci(economy_data[eco_features], economy_data[target_col], label="economy")
bootstrap_prediction_ci(business_data[biz_features], business_data[target_col], label="business")

Bootstrap (economy): 100%|██████████| 100/100 [00:20<00:00,  4.85it/s]



📊 [economy] 統計指標：
   MSE         = 2906795.92
   R^2         = 0.7766
   95% 覆蓋率   = 54.50%
   99% 覆蓋率   = 63.05%

✅ [economy] Bootstrap CI 結果已儲存！


Bootstrap (business): 100%|██████████| 100/100 [00:18<00:00,  5.50it/s]



📊 [business] 統計指標：
   MSE         = 10200824.07
   R^2         = 0.7454
   95% 覆蓋率   = 56.74%
   99% 覆蓋率   = 65.66%

✅ [business] Bootstrap CI 結果已儲存！
